In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

In [27]:
dataset = load_dataset("opus100", "en-tr", split="train") #HuggingFace API anahtarını Colab Gizli Anahtarlar alanına giriyoruz
dataset = dataset.select(range(500))  # Örneğin ilk 500 örneği al


dataset = dataset.train_test_split(test_size=0.1)

train_data = dataset["train"]
test_data = dataset["test"]

print(len(train_data))

for i in range(10):
    example = train_data[i]
    print(example)


450
{'translation': {'en': "He's too big.", 'tr': 'Çünkü ona ulaşılamaz.'}}
{'translation': {'en': '- Thank you, Sheila.', 'tr': '- Teşekkürler, Sheila.'}}
{'translation': {'en': "- You don't have to kill him, Mr Howie.", 'tr': '- Onu öldürmenize gerek yok, Bay Howie.'}}
{'translation': {'en': 'One second.', 'tr': 'Bir saniye.'}}
{'translation': {'en': "I don't feel like swimming.", 'tr': 'Pek canım istemiyor.'}}
{'translation': {'en': 'What a total dick.', 'tr': 'Tam bir pislik.'}}
{'translation': {'en': 'He was already gone.', 'tr': 'Çoktan gitmişti.'}}
{'translation': {'en': '- Chips?', 'tr': '- Chips?'}}
{'translation': {'en': 'Oh!', 'tr': 'Yarış çok çetin.'}}
{'translation': {'en': '- Fucking bitch.', 'tr': '- Lanet olası kaltak!'}}


In [28]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")

def preprocess(example):
    model_inputs = tokenizer(example["translation"]["tr"], max_length=64, truncation=True, padding="max_length")
    labels = tokenizer(example["translation"]["en"], max_length=64, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

train_data = train_data.map(preprocess, remove_columns=["translation"])
test_data = test_data.map(preprocess, remove_columns=["translation"])


Map:   0%|          | 0/450 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

In [29]:
def collate_fn(batch):
    input_ids = torch.tensor([item["input_ids"] for item in batch])
    labels = torch.tensor([item["labels"] for item in batch])
    return input_ids, labels

train_loader = DataLoader(train_data, batch_size=8, shuffle=True, collate_fn=collate_fn)


In [30]:
class TranslateGPT(nn.Module):
    def __init__(self, vocab_size, embed_dim=256, num_heads=4, num_layers=2):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.pos_encoding = nn.Parameter(torch.randn(1, 64, embed_dim))
        self.encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=embed_dim, nhead=num_heads, dim_feedforward=512, dropout=0.1, batch_first=True),
            num_layers=num_layers
        )
        self.decoder = nn.TransformerDecoder(
            nn.TransformerDecoderLayer(d_model=embed_dim, nhead=num_heads, dim_feedforward=512, dropout=0.1, batch_first=True),
            num_layers=num_layers
        )
        self.output_layer = nn.Linear(embed_dim, vocab_size)

    def forward(self, src, tgt):
        src_emb = self.embedding(src) + self.pos_encoding[:, :src.size(1), :]
        tgt_emb = self.embedding(tgt) + self.pos_encoding[:, :tgt.size(1), :]
        memory = self.encoder(src_emb)
        output = self.decoder(tgt_emb, memory)
        return self.output_layer(output)


In [88]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TranslateGPT(vocab_size=tokenizer.vocab_size).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
loss_fn = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)

# İlk 5 örneği al
mini_data = train_data.select(range(5))

mini_loader = DataLoader(mini_data, batch_size=1, shuffle=True, collate_fn=collate_fn)

# Eğitim: 100 epoch (küçük veri)
for epoch in range(100):
    model.train()
    total_loss = 0
    for input_ids, labels in mini_loader:
        input_ids, labels = input_ids.to(device), labels.to(device)
        tgt_input = labels[:, :-1]
        tgt_output = labels[:, 1:]

        logits = model(input_ids, tgt_input)
        loss = loss_fn(logits.reshape(-1, logits.size(-1)), tgt_output.reshape(-1))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    if (epoch + 1) % 10 == 0:
        print(f"[Mini] Epoch {epoch+1}, Loss: {total_loss:.4f}")


[Mini] Epoch 10, Loss: 40.0616
[Mini] Epoch 20, Loss: 24.9593
[Mini] Epoch 30, Loss: 13.5387
[Mini] Epoch 40, Loss: 6.7202
[Mini] Epoch 50, Loss: 3.3147
[Mini] Epoch 60, Loss: 1.7415
[Mini] Epoch 70, Loss: 1.0919
[Mini] Epoch 80, Loss: 0.7402
[Mini] Epoch 90, Loss: 0.5392
[Mini] Epoch 100, Loss: 0.4277


In [90]:
def translate_text(model, input_text, max_len=64, verbose=True):
    model.eval()
    tokens = tokenizer(input_text, return_tensors="pt", padding="max_length", max_length=max_len, truncation=True)
    src = tokens["input_ids"].to(device)

    # Başlangıç tokenı (CLS veya BOS, varsa BOS kullan)
    bos_token = tokenizer.bos_token_id or tokenizer.cls_token_id
    sep_token = tokenizer.sep_token_id
    pad_token = tokenizer.pad_token_id

    tgt = torch.tensor([[bos_token]], dtype=torch.long).to(device)

    for _ in range(max_len):
        output = model(src, tgt)
        next_token = torch.argmax(output[:, -1, :], dim=-1, keepdim=True)

        if next_token.item() in [sep_token, pad_token]:
            break  # optional: veya devam et

        tgt = torch.cat([tgt, next_token], dim=1)

    if verbose:
        print("Output token ids :", tgt[0].tolist())
        print("Raw decoded      :", tokenizer.decode(tgt[0], skip_special_tokens=False))
        print("Cleaned decoded  :", tokenizer.decode(tgt[0], skip_special_tokens=True))

    return tokenizer.decode(tgt[0], skip_special_tokens=True)




In [91]:
# Örnek
sample = "- Onu öldürmenize gerek yok, Bay Howie."
print("Girdi:", sample)
translated = translate_text(model, sample)
print("Çeviri:", translated)

Girdi: - Onu öldürmenize gerek yok, Bay Howie.
Output token ids : [101, 118, 11065, 16938, 112, 188, 10529, 10114, 32099, 10957, 117, 12916, 14962, 10400, 119]
Raw decoded      : [CLS] - You don ' t have to kill him, Mr Howie.
Cleaned decoded  : - You don ' t have to kill him, Mr Howie.
Çeviri: - You don ' t have to kill him, Mr Howie.


ATTENTION MEKANİZMASI NE SAĞLADI

🎯 Odaklanma	Model, hangi kelimenin hangi kelimeyle ilişkili olduğunu öğrenir (örneğin "teşekkürler" → "thank you")

🧩 Parçalı anlam	Tüm cümleyi tek bir gizli vektöre sıkıştırmak yerine, her bir kelimenin bağlamına özgü temsilini tutar

🔀 Doğal kelime sırası	Diller arası kelime sıraları farklı olsa da, attention mekanizması bu farklılıkları çözebilir

🔁 Uzun bağımlılıklar	Cümlenin başındaki bir kelimeyle sonundaki kelime arasındaki ilişkileri kurabilir

EĞİTİMİN AMACI NEDİR, LOOKUP TABLE KULLANILAMAZ MIYDI?

Eğitim sırasında:

Model sadece ezberlemez, genelleştirir

Aynı yapıyı taşıyan ama yeni kelimelerle gelen cümleleri çevirmeyi öğrenir

Örneğin:


Görmedim → I didn't see
Duymadım → I didn't hear
gibi örneklerden sonra, model:

Anlamadım → I didn't understand
gibi çeviriyi daha önce görmemiş olsa bile çıkarabilir.